In [1]:
import pandas as pd
import numpy as np

from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, f1_score, precision_score,\
recall_score, confusion_matrix, classification_report, accuracy_score 
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.linear_model import SGDClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sys import argv
import gc
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from imblearn.pipeline import make_pipeline, Pipeline
from imblearn.combine import SMOTEENN
from sklearn.preprocessing import StandardScaler, Normalizer

In [28]:
path = '/Users/lucia/phd_work/Clpsy/'
features = pd.read_csv(path + 'suicideDetection/features/FreqSentiMotiTopiFea.csv')
#merge with liwc data
liwc = pd.read_csv(path + 'suicideDetection/features/liwcSW.csv')
liwc = liwc.rename(columns = {liwc.columns[2]:'user_id'})
liwcUser = liwc.groupby('user_id').mean().reset_index()
liwcUser = liwcUser.drop(['Source (A)', 'Source (D)'], axis=1)
allfea = pd.merge(features, liwcUser, on = 'user_id', how = 'right')
# merge empath
empath = pd.read_csv(path + 'suicideDetection/features/empathSW.csv')
allfea = pd.merge(allfea, empath, on = 'user_id', how = 'right')
#merge tags
# tags = pd.read_csv(path + 'suicideDetection/features/TagFeaSW.csv')
# allfea = pd.merge(allfea, tags, on = 'user_id', how = 'right')

allfea.shape

(496, 146)

In [75]:
Lmodeltest = pd.read_csv(path + 'data/languageModel/allPtest.csv')
testId = Lmodeltest['user_id']
testId = testId.drop_duplicates(keep='first', inplace=False).to_frame()
X_train = pd.merge(allfea, testId, on = 'user_id', how='right')
y_train = X_train.raw_label 
X_train = X_train.iloc[:, 3:146]
X_train.shape

(248, 143)

In [74]:
#y_train = pd.merge(allfea, testId, on = 'user_id', how='outer')
X_test = allfea[(~allfea.user_id.isin(testId.user_id))]
y_test = X_test.raw_label 
X_test = X_test.iloc[:, 3:146]
X_test.shape


(248, 143)

In [77]:
X = allfea.iloc[:, 3:146]
y = allfea.raw_label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=30)

def SGDclassifier(X_train, X_test, y_train, y_test):
    smote_enn = SMOTEENN(random_state=42)

    clf = make_pipeline(smote_enn, StandardScaler(), SGDClassifier(max_iter= 1000))

    parameters = [{'sgdclassifier__alpha': [0.01, 0.05, 0.001, 0.005], 'sgdclassifier__class_weight':['balanced'],
                  'sgdclassifier__loss': ['hinge','log','modified_huber','squared_hinge', 'perceptron'], 
                   'sgdclassifier__penalty':['none','l1','l2']}]

    grid_search_item = GridSearchCV(clf,
                              param_grid = parameters,
                               scoring = 'accuracy',
                               cv = 5,
                               n_jobs = -1)

    grid_search = grid_search_item.fit(X_train, y_train)

    print('Best scores and best parameters')
    print(grid_search.best_score_)
    print(grid_search.best_params_)

    y_true, y_pred = y_test, grid_search.predict(X_test)
    print(classification_report(y_true, y_pred))
    print(confusion_matrix(y_true, y_pred))

SGDclassifier(X_train, X_test, y_train, y_test)

Best scores and best parameters
0.41935483870967744
{'sgdclassifier__alpha': 0.01, 'sgdclassifier__class_weight': 'balanced', 'sgdclassifier__loss': 'perceptron', 'sgdclassifier__penalty': 'none'}
              precision    recall  f1-score   support

           1       0.62      0.56      0.59        61
           2       0.07      0.25      0.11        20
           3       0.25      0.16      0.19        51
           4       0.58      0.43      0.50       116

   micro avg       0.39      0.39      0.39       248
   macro avg       0.38      0.35      0.34       248
weighted avg       0.48      0.39      0.42       248

[[34 14  7  6]
 [ 4  5  4  7]
 [ 3 17  8 23]
 [14 39 13 50]]


/Users/lucia/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/lucia/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
/Users/lucia/anaconda3/lib/python3.6/site-packages/imblearn/pipeline.py:349: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [79]:
Lmodeltest = pd.read_csv(path + 'data/languageModel/allPtest.csv')
testId = Lmodeltest['user_id']
testId = testId.drop_duplicates(keep='first', inplace=False).to_frame()
X_test = pd.merge(allfea, testId, on = 'user_id', how='right')
y_test = X_test.raw_label 
X_test = X_test.iloc[:, 3:146]
X_train = allfea[(~allfea.user_id.isin(testId.user_id))]
y_train = X_train.raw_label 
X_train = X_train.iloc[:, 3:146]
SGDclassifier(X_train, X_test, y_train, y_test)

/Users/lucia/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/lucia/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


Best scores and best parameters
0.49193548387096775
{'sgdclassifier__alpha': 0.01, 'sgdclassifier__class_weight': 'balanced', 'sgdclassifier__loss': 'modified_huber', 'sgdclassifier__penalty': 'l1'}
              precision    recall  f1-score   support

           1       0.53      0.51      0.52        76
           2       0.16      0.36      0.23        22
           3       0.32      0.24      0.27        55
           4       0.45      0.40      0.42        95

   micro avg       0.40      0.40      0.40       248
   macro avg       0.36      0.38      0.36       248
weighted avg       0.42      0.40      0.40       248

[[39  8  7 22]
 [ 4  8  3  7]
 [12 13 13 17]
 [19 20 18 38]]


/Users/lucia/anaconda3/lib/python3.6/site-packages/imblearn/pipeline.py:349: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [34]:
stressed = pd.read_csv(path + 'data/clpsych19_training_data/Stressed.csv')
StressedSW = SW.append(stressed, ignore_index=True)
StressedSW.to_csv(path + 'data/clpsych19_training_data/StressedSW.csv')


In [35]:
path = '/Users/lucia/phd_work/Clpsy/'
allP = pd.read_csv(path + 'data/clpsych19_training_data/Btrain_NoNoise.csv')
SW= pd.read_csv(path + 'data/clpsych19_training_data/Btrain_NoNoise_SW.csv')
SW= pd.read_csv(path + 'data/clpsych19_training_data/Btrain_NoNoise_SW.csv')

labels = pd.read_csv(path + 'data/clpsych19_training_data/crowd_train.csv')
labels = labels.loc[labels['user_id'] > 0]
x_train,x_test = train_test_split(labels,test_size=0.5)

#get test and train file
def mergeFile(file, testSave):
    filetrain = pd.merge(file, x_train, on ='user_id', how ='right')
    filetest = pd.merge(file, x_test, on ='user_id', how ='right')
    filetest.to_csv()
    return filetrain

testSave1 = path + 'data/languageModel/allPtest.csv'
mergeFile(allP, testSave1)

testSave2 = path + 'data/languageModel/SWPtest.csv'
mergeFile(SW, testSave2)

#SWPtrain = pd.merge(SW, x_train, on ='user_id', how ='right')
#SWPtest = pd.merge(SW, x_test, on ='user_id', how ='right')
#SWPtest.to_csv()

print(len(allP))
print(len(allPtrain))
print(len(allPtest))

print(len(SW))
print(len(SWPtrain))
print(len(SWPtest))

31553
15728
15825
919
516
403


In [5]:
def getClass(file, Class, savePath):
    a = file.loc[file['raw_label_x'] == Class]
    a = a[['post_body','post_title']]
    a.to_csv(savePath)
    

savePath1 = path + 'data/languageModel/AAll.csv'
savePath2 = path + 'data/languageModel/BAll.csv'
savePath3 = path + 'data/languageModel/CAll.csv'
savePath4 = path + 'data/languageModel/DAll.csv'

ClassA = getClass(allPtrain, 'a', savePath1)
ClassB = getClass(allPtrain, 'b', savePath2)
ClassC = getClass(allPtrain, 'c', savePath3)
ClassD = getClass(allPtrain, 'd', savePath4)

In [7]:
savePath1 = path + 'data/languageModel/ASW.csv'
savePath2 = path + 'data/languageModel/BSW.csv'
savePath3 = path + 'data/languageModel/CSW.csv'
savePath4 = path + 'data/languageModel/DSW.csv'

ClassA = getClass(SWPtrain, 'a', savePath1)
ClassB = getClass(SWPtrain, 'b', savePath2)
ClassC = getClass(SWPtrain, 'c', savePath3)
ClassD = getClass(SWPtrain, 'd', savePath4)